### Project - Airline AI Assistant

In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [64]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GEMINI_API_KEY')
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
ollama_url = "http://localhost:11434/v1"

openai = OpenAI()
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
ollama = OpenAI(api_key="ollama", base_url=ollama_url)

In [ ]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence. 
Always be accurate. If you do not know the answer, say so.
"""

In [ ]:
def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    response = ollama.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat).launch()

In [49]:
ticket_prices = {
    "trivandrum": "$1999",
    "halifax": "$2102",
    "london": "$1200",
    "wellington": "$3022",
    "valencia": "$2032",
    "tokyo": "$2031",
    "new york": "$1222",
}

def get_ticket_price(destination_city):
    print(f"getting the ticket price for desitination city : {destination_city}")
    price = ticket_prices.get(destination_city.lower(), "unknown destination. we do not operate to this location")
    return f"The price of a ticket to {destination_city} is {price}"

In [50]:
get_ticket_price("abc")

getting the ticket price for desitination city : abc


'The price of a ticket to abc is unknown destination. we do not operate to this location'

In [51]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to"
            }
        }
    },
    "required": ["destination_city"],
    "additionalProperties": False
}

In [52]:
tools = [{"type": "function", "function": price_function}]
tools

[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': 'Get the price of a return ticket to the destination city.',
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': 'The city that the customer wants to travel to'}}},
   'required': ['destination_city'],
   'additionalProperties': False}}]

In [65]:
def chat_with_tool(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages, tools=tools)
    if response.choices[0].finish_reason == "tool_calls":
        tool_message = response.choices[0].message
        tools_response = handle_tools_call(tool_message)
        messages.append(tool_message)
        messages.append(tools_response)

        for message in messages:
            print(message)

        response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
        
    return response.choices[0].message.content

In [58]:
def handle_tools_call(message):
   tools_call = message.tool_calls[0]
   if tools_call.function.name == "get_ticket_price":
      arguments = json.loads(tools_call.function.arguments)
      city = arguments.get("destination_city")
      price_details = get_ticket_price(city)
      response = {
         "role": "tool",
         "content": price_details,
         "tool_call_id": tools_call.id
      }
   return response

In [ ]:
gr.ChatInterface(fn=chat_with_tool).launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7893
* To create a public link, set `share=True` in `launch()`.


getting the ticket price for desitination city : Trivandrum
{'role': 'system', 'content': '\nYou are a helpful assistant for an Airline called FlightAI.\nGive short, courteous answers, no more than 1 sentence. \nAlways be accurate. If you do not know the answer, say so.\n'}
{'role': 'user', 'content': [{'text': 'hi', 'type': 'text'}]}
{'role': 'assistant', 'content': [{'text': 'Hello! How can I assist you with your flight today?', 'type': 'text'}]}
{'role': 'user', 'content': [{'text': "i'd like to go for a trip", 'type': 'text'}]}
{'role': 'assistant', 'content': [{'text': 'That sounds great! Where would you like to travel?', 'type': 'text'}]}
{'role': 'user', 'content': 'Trivandrum please'}
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_CMK354JHJAiNaejUee6xs7Ze', function=Function(arguments='{"destination_city":"Trivandrum"}', name='get_ticket_price'), type=

Traceback (most recent call last):
  File "/Users/rijuvijayan/1workspace/learning/udemy/LLM_Engineering/.venv/lib/python3.11/site-packages/gradio/queueing.py", line 763, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rijuvijayan/1workspace/learning/udemy/LLM_Engineering/.venv/lib/python3.11/site-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rijuvijayan/1workspace/learning/udemy/LLM_Engineering/.venv/lib/python3.11/site-packages/gradio/blocks.py", line 2125, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rijuvijayan/1workspace/learning/udemy/LLM_Engineering/.venv/lib/python3.11/site-packages/gradio/blocks.py", line 1605, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^

getting the ticket price for desitination city : Wellington
{'role': 'system', 'content': '\nYou are a helpful assistant for an Airline called FlightAI.\nGive short, courteous answers, no more than 1 sentence. \nAlways be accurate. If you do not know the answer, say so.\n'}
{'role': 'user', 'content': [{'text': 'hi', 'type': 'text'}]}
{'role': 'assistant', 'content': [{'text': 'Hello! How can I assist you with your flight plans today?', 'type': 'text'}]}
{'role': 'user', 'content': [{'text': 'i want to go to wellington', 'type': 'text'}]}
{'role': 'assistant', 'content': [{'text': 'I can help with that; would you like to know the price for a return ticket to Wellington?', 'type': 'text'}]}
{'role': 'user', 'content': 'yes'}
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_7bDTvOuGLxFfGrENSDVX4KP5', function=Function(arguments='{"destination_city":"Wellington"}',